In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [3]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient
i=0
def TPN(i):
    PI_i = Graph_Entropy_measure.PI_of(i)
    TC_i = Tr_centrality_measure.TC_of(i)
    CC_i = Constraint_Coefficient.CC_of(i)
    
    TPN_i = (PI_i + TC_i + CC_i)/3
    
    return TPN_i

In [5]:
#loop
def loop_TPN():
    i=1
    list_TPN=[]    
    for i in Selection_Algorithm.sel_subgraphs():          
        TPN_ = TPN(i)
        i+=1
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [ ]:
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1])

#TPN_Rank

[[15, 0.2650181124586119],
 [39, 0.2850802301167281],
 [8, 0.29167291975154785],
 [30, 0.3027780932939072],
 [53, 0.32036010581071367],
 [31, 0.32125947201143196],
 [16, 0.33362990110337526],
 [12, 0.3374868684924472],
 [61, 0.3445011540644978],
 [29, 0.34526242325068773],
 [22, 0.35055351288202546],
 [69, 0.35109420506290734],
 [6, 0.35175134890771736],
 [67, 0.3808798489699579],
 [78, 0.3844549978322802],
 [80, 0.4140304809134446],
 [81, 0.4140304809134446],
 [9, 0.44572537482820357],
 [5, 0.4601564889617687],
 [57, 0.4722189615195925],
 [58, 0.4722189615195925],
 [59, 0.4722189615195925],
 [4, 0.5265671836987228],
 [7, 0.5265671836987228],
 [10, 0.5265671836987228]]

In [ ]:
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
print(Top_Five)
#Top_Five.to_csv('result/TPN_email_network.csv')

In [ ]:
#plt.clf()
#H = G.remove_node(0)
#G.remove_edge(Selection_Algorithm.subgraph_of(33).edges())
#nx.draw(Selection_Algorithm.subgraph_of(79), with_labels = True)
#plt.show()
#plt.savefig("out.png")

In [8]:
#loop_TPN()